This notebook generates several synthetic wind time series meant to replicate the real wind data and then runs the wind profile analysis.

In [3]:
# %matplotlib inline
%load_ext autoreload
%autoreload 2

import matplotlib.pyplot as plt
import numpy as np
from local_utils import *
import matplotlib.dates as md

which_sol = "0133"
path = './MEDA_WS_Data/'
WS_file = path + 'WE__%s___________DER_WS__________________P02.xml' % (which_sol)

time, wind = retrieve_time_wind(WS_file)
wind *= 100. # cm/s
saved_zstar = 10. # cm
correct_ustar = kappa*np.mean(wind)/np.log(zs[0]/saved_zstar)
print(correct_ustar)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Processing label: ./MEDA_WS_Data/WE__0133___________DER_WS__________________P02.xml
Now processing a Header structure: HEADER
Now processing a Table_Delimited structure: TABLE
34.45792737957053


In [ ]:
num_time_series = 10

correct_ustars = np.array([])
saved_ustars = np.array([])
saved_zstars = np.array([])

for i in range(num_time_series):
    synthetic_wind = generate_synthetic_wind(time, wind)
    scaled_windspeeds = create_synthetic_wind_profile(synthetic_wind, zs, correct_zstar, z0=np.min(zs))    
    averaged_windspeeds, std_windspeeds = sample_wind_profile(sample_time, t0, time, scaled_windspeeds, zs)
    
    ax1 = make_plot_of_original_and_scaled_windspeeds(time, synthetic_wind, zs, sample_time, t0, scaled_windspeeds, 
                                                  averaged_windspeeds, ax1)

    ustar, zstar, sigma_ustar, sigma_zstar, std_windspeeds =\
        fit_wind_profile_scaled_values(zs, averaged_windspeeds, std_windspeeds, rescale_unc=False)

    ax2 = make_plot_of_wind_data_and_profile_scaled_values(zs, averaged_windspeeds, 
        std_windspeeds, ustar, zstar, sigma_ustar, sigma_zstar, ax2)

    # Collect ustar and zstar and uncertainties - Check that the fit worked
    correct_ustars = np.append(correct_ustars, kappa*averaged_windspeeds[0]/np.log(zs[0]/correct_zstar))

    saved_ustars = np.append(saved_ustars, ustar)
    saved_zstars = np.append(saved_zstars, zstar)
    saved_sigma_ustars = np.append(saved_sigma_ustars, sigma_ustar)
    saved_sigma_zstars = np.append(saved_sigma_zstars, sigma_zstar)

saved_zstar = 10. # cm
t0 = np.min(time)
ind = retrieve_relevant_times(time, t0, len(zs)*sample_time)
saved_ustar = kappa*np.median(synthetic_wind[ind])/np.log(zs[0]/saved_zstar)

scaled_windspeeds = create_synthetic_wind_profile(synthetic_wind, zs, saved_zstar, z0=np.min(zs))
t0 = np.min(time)
averaged_windspeeds, std_windspeeds = sample_wind_profile(sample_time, t0, time, scaled_windspeeds, zs)

u_star, z_star, sigma_u_star, sigma_z_star, std_windspeeds =\
    fit_wind_profile_scaled_values(zs, averaged_windspeeds, std_windspeeds, rescale_unc=False)

ax2 = make_plot_of_wind_data_and_profile_scaled_values(zs, averaged_windspeeds, 
    std_windspeeds, u_star, z_star, sigma_u_star, sigma_z_star, ax2)

ax2.text(0.05, 0.725, r'Correct $u_\star = %.0f\, {\rm cm\ s^{-1}}$' % (saved_ustar), 
         fontsize=28, transform=ax2.transAxes)
ax2.text(0.05, 0.675, r'Correct $z_\star = %.0f\, {\rm cm}$' % (saved_zstar), 
         fontsize=28, transform=ax2.transAxes)
ax1.set_xlim([np.min(time), np.min(time) + timedelta(seconds=len(zs)*sampling_duration)])